In [ ]:
import pandas as pd

In [ ]:
import re

In [ ]:
import numpy as np

In [101]:
timesData = pd.read_csv("dataframe/timesData.csv")
timesData.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011


In [ ]:
timesData.info()

In [ ]:
timesData['income']=pd.to_numeric(timesData['income'], errors='coerce')

In [ ]:
shanghaiData = pd.read_csv("dataframe/shanghaiData.csv")
shanghaiData.head()

In [ ]:
school_and_country = pd.read_csv("dataframe/school_and_country_table.csv")
school_and_country.head()

In [ ]:
cwurData = pd.read_csv("dataframe/cwurData.csv")
cwurData.head()

In [ ]:
educational_attainment = pd.read_csv("dataframe/educational_attainment_supplementary_data.csv")
educational_attainment.head()

In [ ]:
education_expenditure = pd.read_csv("dataframe/education_expenditure_supplementary_data.csv",engine='python')
education_expenditure.head()

1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframe

In [ ]:
most_recent_data=timesData.loc[(timesData.groupby('university_name')['year'].idxmax())]
most_recent_data.head()

In [ ]:
least_recent_data=timesData.loc[(timesData.groupby('university_name')['year'].idxmin())]
least_recent_data.head()

In [ ]:
##timesData[timesData['year']>=timesData['year'].max()]

In [ ]:
##timesData[timesData['university_name']=='Aalto University']

In [ ]:
#timesData.groupby('university_name').min()['year']


In [ ]:
#timesData.groupby('university_name').max()['year']

2. For each university, compute the improvement in "income" between the least recent and the most recent data points

In [ ]:
punto2=pd.merge(most_recent_data, least_recent_data, on='university_name')
punto2.head()

In [ ]:
punto2.columns

In [ ]:
##nuovo_punto2=punto2[punto2[['income_x','income_y']].notnull()]
##nuovo_punto2.head()

In [ ]:
##def calcola_perc(riga):
    ##return riga['income_x'] - riga['income_y']

In [ ]:
##nuovo_punto2['income_improvement']=nuovo_punto2.apply(calcola_perc, axis=1)
##nuovo_punto2.head()

In [ ]:
##nuovo_punto2['differenza']=nuovo_punto2['income_x']-nuovo_punto2['income_y']
##nuovo_punto2

In [ ]:
#nuovo_punto2[nuovo_punto2['income_y']-nuovo_punto2['income_x']]

In [ ]:
##lista=nuovo_punto2['income_x']
##lista


In [ ]:
##nuovo=lista.astype(float)
##nuovo

In [ ]:
def calcola_perc(riga):
    return riga['votes_first_vote']/ risultato[riga['state']] *100

In [ ]:
##nuovo_punto2['income_y']=nuovo_punto2['income_y'].to_numeric()

In [ ]:
##nuovo_punto2=np.array(list, dtype=float)

In [ ]:
punto2['income_improvement']=punto2['income_y']-punto2['income_x']
punto2.head()

3. Find the university with the largest increase computed in the previous point

In [ ]:
punto2.loc[punto2['income_improvement']>= punto2['income_improvement'].max()]['university_name'] #possiamo anche togliere l'ultima quadra


4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

In [ ]:
cwurData[cwurData['institution']=='Aarhus University']

In [ ]:
shanghaiData[shanghaiData['university_name']=='Aarhus University']

5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [ ]:
most_recent_data.head()

In [ ]:
most_recent_data['year']

In [ ]:
most_recent_data

In [ ]:
rapporto=most_recent_data['female_male_ratio']
rapporto.head()

In [ ]:
dataframe=pd.DataFrame(rapporto)
dataframe.head()

In [ ]:
most_recent_data.info()

In [ ]:
most_recent_data['female'] = most_recent_data['female_male_ratio'].str.extract(r'(^\d+)')
most_recent_data['male'] = most_recent_data['female_male_ratio'].str.extract(r'(\d+$)')
most_recent_data.head()

In [ ]:
most_recent_data['male']=pd.to_numeric(most_recent_data['male'], errors='coerce')
most_recent_data['female']=pd.to_numeric(most_recent_data['female'], errors='coerce')

In [ ]:
most_recent_data.info()

In [ ]:
most_recent_data['num_students'] = most_recent_data['num_students'].str.replace(',','')
most_recent_data.head()

In [ ]:
most_recent_data['num_students']=pd.to_numeric(most_recent_data['num_students'], errors='coerce')
most_recent_data.head()

In [ ]:
##def calc_numero(row):
    ##return most_recent_data['num_students']* most_recent_data[row]//100

In [ ]:
#most_recent_data['num_male']=most_recent_data['male'].apply(calc_numero, axis= 1)

In [ ]:
most_recent_data['num_male']= most_recent_data['male'] * most_recent_data['num_students'] // 100
most_recent_data.head()

In [ ]:
most_recent_data['num_fem']= most_recent_data['female'] * most_recent_data['num_students'] // 100
most_recent_data.head()

6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [ ]:
most_recent_data['rapporto'] = most_recent_data['num_fem'] / most_recent_data['num_male']
media = most_recent_data['rapporto'].notnull().mean()

In [ ]:
punto6 = most_recent_data.loc[most_recent_data['rapporto'] <= media]
punto6.head()

In [ ]:
punto6['university_name']

7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

In [ ]:
#denominatore
deno = most_recent_data.groupby('country', as_index = True)['num_students'].sum()
#numeratore
deno

In [ ]:
num=punto6.groupby('country')['num_students'].sum()

In [ ]:
num.dtype

In [ ]:
deno.dtype

In [ ]:
fraction=num/deno
fraction

In [ ]:
#most_recent_data.loc[most_recent_data.groupby('country').sum()['num_students']]

8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [ ]:
educational_attainment = pd.read_csv("dataframe/educational_attainment_supplementary_data.csv")
educational_attainment.head()

In [ ]:
#educational_attainment.dropna(educational_attainment,how="any", thresh=None, subset=(['country_name','series_name']), inplace=True)
#educational_attainment.head() 

In [ ]:
educ = educational_attainment[(educational_attainment['country_name'].notnull()) | (educational_attainment['series_name'].notnull())]
educ.head()

In [ ]:
#educational_attainment.dropna?


9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [ ]:
ed_pulito= educ.dropna(axis='columns', how='all')
ed_pulito.head()

In [ ]:
#ed_pulito[ed_pulito['1986'].notnull()]

In [ ]:
pd.melt?

In [ ]:
ed_pulito.columns[2:]

In [ ]:
pd.melt(ed_pulito, id_vars=['country_name','series_name'], value_vars=['1985', '1986', '1987', '1990', '1991', '1992', '1993', '1995', '1996',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013'])


In [ ]:
ed_pulito.transpose?

10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [ ]:
#tab=pd.merge(timesData, shanghaiData, on='university_name')
#tab_merge=pd.merge(tab, cwurData, left_on='university_name', right_on='institution')
#tab_merge

In [ ]:
tabella_merge=pd.merge(school_and_country, timesData, left_on='school_name', right_on='university_name', how='left')
tabella_merge.loc[tabella_merge['university_name'].isnull()]

In [ ]:
tabella_merge=pd.merge(school_and_country, shanghaiData, left_on='school_name', right_on='university_name', how='left')
tabella_merge.head()

In [ ]:
tabella_merge.loc[tabella_merge['university_name'].isnull()]

In [ ]:
tabella_merge3=pd.merge(tabella_merge, cwurData, left_on='school_name', right_on='institution', how='left')
tabella_merge3.head()

In [ ]:
tabella_merge3.columns

In [ ]:
punto10=tabella_merge3.loc[(tabella_merge3['institution'].isnull())|(tabella_merge3['university_name'].isnull())][['university_name','institution','school_name']]


In [ ]:
punto10.loc[(punto10.groupby('school_name')['institution'].unique())]

In [ ]:
#pd.punto10(['school_name'], index=False).unique()
pd.unique(punto10['school_name'])
#punto10.['school_name'].unique

11. In the times ranking, compute the number of times each university appears

In [109]:
punto11 = timesData.groupby('university_name', as_index = False)['world_rank'].count()
punto11

,university_name,world_rank
0,AGH University of Science and Technology,1
1,Aalborg University,5
2,Aalto University,5
3,Aarhus University,6
4,Aberystwyth University,5
...,...,...
813,École Normale Supérieure,6
814,École Normale Supérieure de Lyon,6
815,École Polytechnique,6
816,École Polytechnique Fédérale de Lausanne,6


12. Find the universities that appear at most twice in the times ranking.

In [114]:
punto12 = punto11[punto11['world_rank']<= 2]
punto12

,university_name,world_rank
0,AGH University of Science and Technology,1
5,Adam Mickiewicz University,1
6,Aix-Marseille University,1
7,Ajou University,1
9,Alexandru Ioan Cuza University,1
...,...,...
806,Yokohama City University,1
807,Yokohama National University,1
810,Yuan Ze University,2
811,Yıldız Technical University,1


13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

4-10-13